## Завдання
В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням.

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статті.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

In [52]:
import os
import time
import pandas as pd
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [53]:
# Шлях до датасету та вихідного файлу
data_folder = '/content/gdrive/MyDrive/Data_science/hw_05/data'
output_file = '/content/gdrive/MyDrive/Data_science/hw_05/combined_data.csv'

In [54]:
# Створюємо порожній DataFrame для зберігання даних
combined_data = pd.DataFrame()

In [55]:
# Список активностей
activities = ['walking', 'running', 'idle', 'stairs']

In [56]:
# Проходимося по кожній папці з даними (walking, running, idle, stairs)
for activity in activities:
    activity_folder = os.path.join(data_folder, activity)

    # Створюємо порожній DataFrame для обраного класу
    activity_data = pd.DataFrame()

    # Проходимося по кожному файлу CSV в папці
    for file_name in os.listdir(activity_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(activity_folder, file_name)

            # Читаємо дані з файлу
            df = pd.read_csv(file_path)

            # Перевіряємо наявність NaN значень у даному файлі
            nan_count = df.isna().sum().sum()
            if nan_count > 0:
                print(f"У файлі {file_path} є {nan_count} NaN значень.")

            # Додаємо дані з файлу до DataFrame класу
            activity_data = pd.concat([activity_data, df], ignore_index=True)

    # Додаємо колонку 'activity' з інформацією про активність
    activity_data['activity'] = activity

    # Додаємо дані класу до загального DataFrame
    combined_data = pd.concat([combined_data, activity_data], ignore_index=True)

# Зберігаємо об'єднані дані в один CSV-файл
combined_data.to_csv(output_file, index=False)

In [57]:
# Перевіряємо кількість NaN значень
nan_count_total = combined_data.isna().sum().sum()
print(f"Кількість значень NaN в об'єднаних даних: {nan_count_total}")

Кількість значень NaN в об'єднаних даних: 0


In [58]:
# Завантаження об'єднаного датасету
combined_data = pd.read_csv('/content/gdrive/MyDrive/Data_science/hw_05/combined_data.csv')

In [59]:
# Розділення на ознаки (X) та цільову змінну (y)
X = combined_data[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
y = combined_data['activity']

In [60]:
# Розділення на навчальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
# Модель SVM
start_time = time.time()
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_elapsed_time = time.time() - start_time

In [62]:
# Модель випадкового лісу
start_time = time.time()
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_elapsed_time = time.time() - start_time

In [63]:
# Оцінка результатів для моделі SVM
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_report = classification_report(y_test, svm_predictions)
svm_confusion = confusion_matrix(y_test, svm_predictions)

# Оцінка результатів для моделі випадкового лісу
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_report = classification_report(y_test, rf_predictions)
rf_confusion = confusion_matrix(y_test, rf_predictions)

In [64]:
# Вивід результатів
print("Результати для SVM:")
print(f"Точність: {svm_accuracy}")
print(f"Звіт про класифікацію:\n{svm_report}")
print(f"Матриця плутанини:\n{svm_confusion}")
print(f"Час виконання SVM: {svm_elapsed_time} сек.")

print("\nРезультати для випадкового лісу:")
print(f"Точність: {rf_accuracy}")
print(f"Звіт про класифікацію:\n{rf_report}")
print(f"Матриця плутанини:\n{rf_confusion}")
print(f"Час виконання випадкового лісу: {rf_elapsed_time} сек.")


Результати для SVM:
Точність: 0.8966522232538946
Звіт про класифікацію:
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      6221
     running       0.93      0.91      0.92     20533
      stairs       1.00      0.01      0.01       915
     walking       0.81      0.90      0.85     11103

    accuracy                           0.90     38772
   macro avg       0.92      0.70      0.69     38772
weighted avg       0.90      0.90      0.89     38772

Матриця плутанини:
[[ 6138    61     0    22]
 [  212 18600     0  1721]
 [    9   223     5   678]
 [   32  1049     0 10022]]
Час виконання SVM: 484.9281554222107 сек.

Результати для випадкового лісу:
Точність: 0.9997936655318271
Звіт про класифікацію:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6221
     running       1.00      1.00      1.00     20533
      stairs       1.00      0.99      1.00       915
     walking       1

## Аналіз результатів:
Звіт про класифікацію надає інформацію про precision, recall та F1-score для кожного класу окремо. Ось значення цих метрик для кожного класу:

### Для SVM:

Для класу "idle":
precision: 0.96
recall: 0.99
F1-score: 0.97

Для класу "running":
precision: 0.93
recall: 0.91
F1-score: 0.92

Для класу "stairs":
precision: 1.00
recall: 0.01
F1-score: 0.01

Для класу "walking":
precision: 0.81
recall: 0.90
F1-score: 0.85

### Для випадкового лісу:

Для класу "idle":
precision: 1.00
recall: 1.00
F1-score: 1.00

Для класу "running":
precision: 1.00
recall: 1.00
F1-score: 1.00

Для класу "stairs":
precision: 1.00
recall: 0.99
F1-score: 1.00

Для класу "walking":
precision: 1.00
recall: 1.00
F1-score: 1.00

З цих даних видно, що випадковий ліс має високі precision, recall і F1-score для всіх класів, у той час як SVM має нижчий recall для класу "stairs". Модель випадкового лісу є більш точною для цих даних за цими метриками.